In [1]:
import numpy as np
import awkward
import uproot_methods

JaggedWithLorentz = awkward.Methods.mixin(uproot_methods.classes.TLorentzVector.ArrayMethods, awkward.JaggedArray)

In [2]:
nevents = 50000
np.random.seed(42)

counts = np.minimum(np.random.exponential(2, size=nevents).astype(int), 20)
make_jagged = lambda a: awkward.JaggedArray.fromcounts(counts, a)

nobj = np.sum(counts)
px = np.random.normal(0., 20., size=nobj)
py = np.random.normal(0., 20., size=nobj)
pz = np.sqrt(px**2+py**2)*np.sinh(np.random.uniform(-2.5, 2.5, size=nobj))
pE = np.sqrt(px*px+py*py+pz*pz+0.000511**2)
p4 = JaggedWithLorentz.fromcounts(counts, uproot_methods.TLorentzVectorArray(px, py, pz, pE))

q = make_jagged(np.random.randint(2, size=nobj)*2-1)
idvar = make_jagged(np.random.normal(size=nobj))

# structured style
objects = make_jagged(awkward.Table({
        'p4': p4.content,
        'q': q.content,
        'idvar': idvar.content,
    }))

In [6]:
# create candidates via independent columnar operations
p4_pairs = p4.pairs(same=False)
q_pairs = q.pairs(same=False)
idvar_pairs = idvar.pairs(same=False)
good = (idvar_pairs._0 > 0.1) & (idvar_pairs._1 > 0.1) & (q_pairs._0*q_pairs._1 == -1)
zp4 = p4_pairs._0 + p4_pairs._1
zp4.mass[good]

<JaggedArray [[] [46.62236793 44.48717172] [] ... [12.89840954] [] []] at 7f3ab445ef10>

In [5]:
# create candidates from structure
zcands = objects.pairs(same=False)
good = (zcands._0['idvar'] > 0.1) & (zcands._1['idvar'] > 0.1) & (zcands._0['q']*zcands._1['q'] == -1)
zp4 = zcands._0['p4'] + zcands._1['p4']
zp4.mass[good]

<JaggedArray [[] [46.62236793 44.48717172] [] ... [12.89840954] [] []] at 7f3ab4455d90>

In [17]:
dummy = uproot_methods.TLorentzVector(0.,0.,0.,0.)
(p4.pairs()._0+dummy).mass

<JaggedArray [[] [0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.000511   0.00051099] [0.000511 0.000511 0.000511] ... [0.000511 0.000511 0.000511] [0.000511] [0.000511   0.000511   0.000511   0.00051101 0.00051101 0.000511  ]] at 7f3ab35b27d0>